In [37]:
!pip install boto3 langchain langchain_community -U

  Using cached boto3-1.34.64-py3-none-any.whl.metadata (6.6 kB)
  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.28-py3-none-any.whl.metadata (8.3 kB)
  Using cached botocore-1.34.64-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_core-0.1.32-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.27-py3-none-any.whl.metadata (13 kB)
  Using cached orjson-3.9.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
Using cached boto3-1.34.64-py3-none-any.whl (139 kB)
Using cached langchain-0.1.12-py3-none-any.whl (809 kB)
Using cached langchain_community-0.0.28-py3-none-any.whl (1.8 MB)
Using cached botocore-1.34.64-py3-none-any.whl (12.0 MB)
Using cached langchain_core-0.1.32-py3-none-any.whl (260 kB)
Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
Using cached langsmith-0.1.27-py3-none-a

In [1]:
import json
import boto3
import random

In [2]:
bedrock_runtime = boto3.client('bedrock-runtime')

In [3]:
filename = 'data_result.csv'
# df = pd.read_csv('data_result.csv',nrows=1000)
with open(filename) as f:
    lines = f.readlines()

In [4]:
# 设置要选取的连续元素的长度
n = 10

# 随机选取一个起始索引
start_idx = random.randint(0, len(lines) - n)
lines[start_idx:start_idx+n]

['10217292369193|80|2024-03-01 00:12:32,069|{"uid":"10215984336169","c":1,"custom":"{\\"mod\\":\\"\\",\\"modPic\\":0,\\"officialTitle\\":-1,\\"clientTime\\":\\"170922315110217292369193\\",\\"responseData\\":null,\\"atData\\":null,\\"modid\\":0}","llm":"","m":"看看新英雄，再说","group":""}\n',
 '10833625353658|55|2024-03-01 00:12:32,014|{"uid":"","c":2,"custom":"{\\"mod\\":\\"\\",\\"modPic\\":0,\\"officialTitle\\":-1,\\"clientTime\\":\\"170922315010833625353658\\",\\"responseData\\":null,\\"atData\\":null,\\"modid\\":0}","llm":"","m":"天牧有用吗空军？","group":""}\n',
 '745723672552|100|2024-03-01 00:12:32,778|{"uid":"819098683576","c":1,"custom":"{\\"mod\\":\\"\\",\\"modPic\\":0,\\"officialTitle\\":-1,\\"clientTime\\":\\"1709223152745723672552\\",\\"responseData\\":null,\\"atData\\":null,\\"modid\\":0}","llm":"","m":"第二也行了","group":""}\n',
 '10568238501250|80|2024-03-01 00:12:32,048|{"uid":"10545092103554","c":1,"custom":"{\\"mod\\":\\"\\",\\"modPic\\":0,\\"officialTitle\\":2,\\"clientTime\\":\\"17092

In [91]:
import re
extracted_data = []

# 提取聊天的正则表达式模式
pattern = r'"m":"(.*?)"'

# 过滤出link

# 遍历每一行
for line in lines:
    match = re.search(pattern, line)
    if match:
        text = match.group(1)
        if not text.startswith('link'):
            extracted_data.append(text)

In [92]:
len(extracted_data)

3921373

In [95]:

# 随机选取一个起始索引
def random_cut_arr(arr,n,seed=1):
    random.seed(seed)
    start_idx = random.randint(0, len(arr) - n)
    return arr[start_idx:start_idx+n]

In [96]:
random_cut_arr(extracted_data,10)

['二大爺好！',
 'قول',
 'morning All 👋',
 "I'm starting to understand",
 'Die spinnen doch',
 '😂😂😂😂😂😂😂😂',
 '去群里问问咯',
 'not towards our server',
 '十二点睡 三点醒?',
 '2221的傻狍子，快打宝藏，我劫一车']

In [97]:
cut_extracted_data = random_cut_arr(extracted_data,500)
text = "\n".join(cut_extracted_data)

In [98]:
len(text)

12814

In [99]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate

In [233]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [291]:
template_extract = \
"""You are an expert research assistant, tasked with identifying player sentiments regarding certain in-game items, neutral NPCs, and game market activities. 

Here is a document you will analyze
<doc>
{context}
</doc>

Here is a task:
First, find the quotes from the document that are most relevant to {topic}, and then print them in numbered order. Quotes should be relatively short.
If there are no relevant quotes, write "No relevant quotes" instead.
please enclose your analysis results in xml tag <response>.

for example:
<response>
1. "拍卖行多香" 
2. "我拍到好东西了" 
3. "拍卖行太差劲了" 
4. "auction sucks" 
5. "拍卖行有人发包" 
</response>

Skip the preamble, go straight into the answer.
"""

In [320]:
template_sentiment = \
"""You are a chat message sentiment classifer

Here is a document you will classify the senetiment
<doc>
{relevant_info}
</doc>


please list all the content if it is relevant to {topic} and classify the sentiment of each content into [positive,neutral,negative]'

Please follow below requirements:
1. You will strictly be based on the document in <doc>.
2. please enclose your analysis results in xml tag <sentiment>.

for example:
<sentiment>
1. "拍卖行多香" [positive]
2. "我拍到好东西了" [positive]
3. "拍卖行太差劲了" [negative]
4. "auction sucks" [negative]
5. "拍卖行有人发包" [neutral]
</sentiment>

Skip the preamble, go straight into the answer. 

"""

In [367]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<response>(.*?)</response>"
        match = re.search(pattern, content, re.DOTALL)
        if match:
            text = match.group(1)
            text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
            return text
        else:
            return 'No relevant quotes'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"


class CustOuputParser2(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<sentiment>(.*?)</sentiment>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'No sentiment'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    

In [368]:
def split_into_chunks(arr, n):
    chunks = []
    for i in range(0, len(arr), n):
        chunks.append(arr[i:i+n])
    return chunks

In [369]:
chunks = split_into_chunks(extracted_data,2000)

In [370]:
print(f"total chunks {len(chunks)}")

total chunks 1961


In [371]:
text =  "\n".join(chunks[45])
len(text)

33601

In [372]:
# from langchain_core.output_parsers import StrOutputParser
# prompt = ChatPromptTemplate.from_template(template)
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_sentiment = ChatPromptTemplate.from_template(template_sentiment)

In [373]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
## 如果没有相关内容则无须invoke chain2
def route(info):
    if not 'no relevant quotes' in info['relevant_info'].lower():
        return chain_2

In [374]:
output_parser = CustOuputParser()
output2_parser  = CustOuputParser2()

chain_1 = prompt_extract | llm_sonnet |output_parser
chain_2 = prompt_sentiment | llm_sonnet|output2_parser
# chain_3 = ({'relevant_info':chain_1,'topic':itemgetter('topic')})|prompt_sentiment | llm_haiku|output2_parser


In [375]:
##测试chain3
# chain_3.invoke({'topic':"auction house",
#                        'context':text})

In [376]:
##测试加入了路由选择的chainfull
full_chain = ({'relevant_info':chain_1,'topic':itemgetter('topic')})| RunnableLambda(route)
answer = full_chain.invoke({'topic':"auction house",'context':text})

<response>
1. "有人急眼了@[CTGC] 💕㊖Never" (Someone is legit bidding)
2. "我拿死工资的人" (I'm putting your a ss on my front door)
3. "有红包，50钻" (There is a red packet, 50 gems)
4. "蓝色的" (A blue one)
5. "谁发的" (Who sent it)
6. "我拷，我没拿到" (I didn't get it)
7. "点太快，好像是20钻石" (Clicked too fast, seems like 20 gems)
8. "这是战争" (This is war)
9. "看到了" (I saw it)
10. "有大佬出手了" (Some whale spent money)
</response>
<sentiment>
1. "有人急眼了@(CTGC) 💕㊖Never" [neutral]
2. "我拿死工资的人" [negative]
3. "有红包，50钻" [positive]
4. "蓝色的" [neutral]
5. "谁发的" [neutral]
6. "我拷，我没拿到" [negative]
7. "点太快，好像是20钻石" [neutral]
8. "这是战争" [neutral]
9. "看到了" [neutral]
10. "有大佬出手了" [positive]
</sentiment>

In [377]:
answer

'\n1. "有人急眼了@(CTGC) 💕㊖Never" [neutral]\n2. "我拿死工资的人" [negative]\n3. "有红包，50钻" [positive]\n4. "蓝色的" [neutral]\n5. "谁发的" [neutral]\n6. "我拷，我没拿到" [negative]\n7. "点太快，好像是20钻石" [neutral]\n8. "这是战争" [neutral]\n9. "看到了" [neutral]\n10. "有大佬出手了" [positive]\n'

In [378]:
# question = "is there any content relevant to auction house?"

#'please list all the content if it is relevant and classify the sentiment of each content into [positive,neutral,negative]'

# answer = chain_2.invoke({'topic':"auction house",
#                        'context':text})

In [379]:
# print(answer)

In [382]:
def extract_text_and_sentiment(text) -> list:
    pattern = r'^(.*?)\s*\[(.*?)\]'
    # pattern = r'"(.*?)"\s*\[(.*?)\]$'
    if not text:
        return []
    result = []
    for line in text.split('\n'):
        match = re.match(pattern, line)
        if match:
            # print(match)
            text_part = match.group(1)
            quoted_texts = re.findall(r'"(.*?)"', text_part)
            # print(quoted_texts)
            sentiment_part = match.group(2)
            result.append((quoted_texts[0], sentiment_part))
    return result

In [383]:
extract_text_and_sentiment(answer)

[('有人急眼了@(CTGC) 💕㊖Never', 'neutral'),
 ('我拿死工资的人', 'negative'),
 ('有红包，50钻', 'positive'),
 ('蓝色的', 'neutral'),
 ('谁发的', 'neutral'),
 ('我拷，我没拿到', 'negative'),
 ('点太快，好像是20钻石', 'neutral'),
 ('这是战争', 'neutral'),
 ('看到了', 'neutral'),
 ('有大佬出手了', 'positive')]

In [348]:
#save a list to a local file, if the file is exist then append the list to the file
def save_list(list,file_name):
    with open(file_name,'a') as f:
        for item in list:
            f.write("%s\n" % item)
    f.close()
    return True

In [349]:
#save a list to a csv file using pandas, if the file is exist then append the list to the file
import pandas as pd
import os
def save_to_csv(data, filename):
    new_data = pd.DataFrame(data, columns=['text','sentiment'])
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        #append data to df
        df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data
    df.to_csv(filename, index=False)
    return df

循环每个chunk进行输出,每次追加到csv文件保存，

In [385]:
import time
all_result = []
filename = 'parsed_result'

##断点跳过
skip_num =238
for i,chunk in enumerate(chunks[:1000]):
    if i < skip_num:
        continue
    t1 = time.time()
    print(f'--------chunk idx:{i}-------')
    text =  "\n".join(chunk)
    answer = full_chain.invoke({'topic':"auction house",
                       'context':text})
    extract_ret = extract_text_and_sentiment(answer)
    print(f'\nextract_ret:\n{extract_ret}')
    time.sleep(2)
    print(f'---time cost {time.time()-t1} s -----\nsleep 10s for token tpm\n\n')
    if extract_ret:
        all_result+=extract_ret
        save_to_csv(extract_ret,filename+f"chunk_{i}.csv")
        save_to_csv(extract_ret,filename+f"_all.csv")

--------chunk idx:238-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 9.50108289718628 s -----
sleep 10s for token tpm


--------chunk idx:239-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 8.842019081115723 s -----
sleep 10s for token tpm


--------chunk idx:240-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 8.99118947982788 s -----
sleep 10s for token tpm


--------chunk idx:241-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 9.16816759109497 s -----
sleep 10s for token tpm


--------chunk idx:242-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 9.299543857574463 s -----
sleep 10s for token tpm


--------chunk idx:243-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---time cost 9.416553974151611 s -----
sleep 10s for token tpm


--------chunk idx:244-------
<response>
No relevant quotes
</response>
extract_ret:
[]
---t

KeyboardInterrupt: 

<response>
No relevant quotes
</response>

Error in StreamingStdOutCallbackHandler.on_llm_new_token callback: ZMQError('Socket operation on non-socket')
--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/langchain_core/callbacks/manager.py", line 258, in handle_event
    event = getattr(handler, event_name)(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/langchain_core/callbacks/streaming_stdout.py", line 33, in on_llm_new_token
    sys.stdout.write(token)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/iostream.py", line 694, in write
    self._schedule_flush()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/iostream.py", line 590, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(b"")
  File "/opt/conda/lib/python3.10/site-packages/zmq/sugar/socket.py", line 696, in send
    return super().send(data,

In [144]:
len(all_result)

92

In [117]:
test = \
"""
1. "❤️Holt euch im Auktionshaus die Geschenke ab ❤️" [positive] - This suggests getting gifts from the auction house, which is a positive sentiment.

2. "Under hot deals." [neutral] - This is a neutral statement referring to the "Hot Deals" section in the auction house.

3. "I don't see a lot of things being bid on in the auction house…" [negative] - This expresses a negative sentiment about not seeing many items being bid on in the auction house.

4. "Titan Gear auction" [neutral] - This is a neutral mention of a Titan Gear auction.

5. "有空了带一下惧星" [neutral] - This is a neutral request to be helped with something related to the "Ark" content, which may involve the auction house.
"""

In [120]:
def extract_text_and_sentiment(text) -> list:
    pattern = r'^(.*?)\s*\[(.*?)\]'
    # pattern = r'"(.*?)"\s*\[(.*?)\]$'
    result = []
    for line in text.split('\n'):
        match = re.match(pattern, line)
        if match:
            text_part = match.group(1)
            quoted_texts = re.findall(r'"(.*?)"', text_part)
            sentiment_part = match.group(2)
            result.append((quoted_texts[0], sentiment_part))
    return result

In [121]:
extract_text_and_sentiment(test)

[('❤️Holt euch im Auktionshaus die Geschenke ab ❤️', 'positive'),
 ('Under hot deals.', 'neutral'),
 ("I don't see a lot of things being bid on in the auction house…",
  'negative'),
 ('Titan Gear auction', 'neutral'),
 ('有空了带一下惧星', 'neutral')]